In [25]:
import pandas as pd
import numpy as np
from io import StringIO
Train = pd.read_csv("Data/train.csv")
Test = pd.read_csv("Data/test.csv")

In [23]:
import pandas as pd   
from matplotlib import pyplot
from numpy import mean
import time


from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

import warnings 
warnings.filterwarnings('ignore')

In [26]:
Train.drop('gender', axis=1, inplace=True)
Test.drop('gender', axis=1, inplace=True)
Train.drop('apache_3j_bodysystem', axis=1, inplace=True)
Test.drop('apache_3j_bodysystem', axis=1, inplace=True)
import matplotlib.pyplot as plt

#normalise Train['pre_icu_los_days']
Train['pre_icu_los_days'] = np.log1p(Train['pre_icu_los_days'])
#make hsitogram of normlaised Train['pre_icu_los_days']

#normalise Train['pre_icu_los_days']
Test['pre_icu_los_days'] = np.log1p(Test['pre_icu_los_days'])
#make hsitogram of normlaised Train['pre_icu_los_days']
#simialrly normalise apache_2_diagnosis
Train['apache_2_diagnosis'] = np.log1p(Train['apache_2_diagnosis'])
Test['apache_2_diagnosis'] = np.log1p(Test['apache_2_diagnosis'])
#do for apache_3j_diagnosis as well for both train and test
Train['apache_3j_diagnosis'] = np.log1p(Train['apache_3j_diagnosis'])
#do for test as well
Test['apache_3j_diagnosis'] = np.log1p(Test['apache_3j_diagnosis'])
#make hsitogram of normlaised Train['pre_icu_los_days']
#do for resprate_apache as well
Train['d1_resprate_max'] = np.log1p(Train['d1_resprate_max'])
#make hsitogram of normlaised Train['pre_icu_los_days']
#do for test as well
Test['d1_resprate_max'] = np.log1p(Test['d1_resprate_max'])
#make hsitogram of normlaised Train['pre_icu_los_days']
#do for temp_apache as well
Train['temp_apache'] = np.log1p(Train['temp_apache'])
#do for test as well
Test['temp_apache'] = np.log1p(Test['temp_apache'])
#do foor d1_resprate_max as well
Train['d1_resprate_max'] = np.log1p(Train['d1_resprate_max'])
#do for test as well
Test['d1_resprate_max'] = np.log1p(Test['d1_resprate_max'])
#do for d1_temp_min
Train['d1_temp_min'] = np.log1p(Train['d1_temp_min'])
#do for test as well
Test['d1_temp_min'] = np.log1p(Test['d1_temp_min'])
#do foe h1_spo2_max
Train['h1_spo2_max'] = np.log1p(Train['h1_spo2_max'])
#do for test as well
Test['h1_spo2_max'] = np.log1p(Test['h1_spo2_max'])
#do for h1_spo2_min
Train['h1_spo2_min'] = np.log1p(Train['h1_spo2_min'])
#do for test as well
Test['h1_spo2_min'] = np.log1p(Test['h1_spo2_min'])
#do for d1_glucose_max
Train['d1_glucose_max'] = np.log1p(Train['d1_glucose_max'])
#do for test as well
Test['d1_glucose_max'] = np.log1p(Test['d1_glucose_max'])
#since all these r heavily skewed andaffected by the outliers we will fill them using median imputation

# fill null values with median for temp_apache
Train['temp_apache'].fillna(Train['temp_apache'].median(), inplace=True)

# fill null values with median for d1_potassium_max
Train['d1_potassium_max'].fillna(Train['d1_potassium_max'].median(), inplace=True)

# fill null values with median for apache_4a_hospital_death_prob
Train['apache_4a_hospital_death_prob'].fillna(Train['apache_4a_hospital_death_prob'].median(), inplace=True)

# fill null values with median for apache_4a_icu_death_prob
Train['apache_4a_icu_death_prob'].fillna(Train['apache_4a_icu_death_prob'].median(), inplace=True)
#since all these r heavily skewed andaffected by the outliers we will fill them using median imputation

Test['temp_apache'].fillna(Test['temp_apache'].median(), inplace=True)

# fill null values with median for d1_potassium_max
Test['d1_potassium_max'].fillna(Test['d1_potassium_max'].median(), inplace=True)

# fill null values with median for apache_4a_hospital_death_prob
Test['apache_4a_hospital_death_prob'].fillna(Test['apache_4a_hospital_death_prob'].median(), inplace=True)

# fill null values with median for apache_4a_icu_death_prob
Test['apache_4a_icu_death_prob'].fillna(Test['apache_4a_icu_death_prob'].median(), inplace=True)

# group the dataframe by apache_2_bodysystem and calculate the mean age for each group
mean_age_by_bodysystem = Train.groupby('apache_2_bodysystem')['age'].mean()

# define a function that takes a row of the dataframe as input and returns the mean age of the corresponding apache_2_bodysystem
def fill_age(row):
    if pd.isnull(row['age']):
        if pd.isnull(row['apache_2_bodysystem']):
            return np.nan
        else:
            return mean_age_by_bodysystem[row['apache_2_bodysystem']]
    else:
        return row['age']

# apply the function to each row of the dataframe and fill the missing age values with the corresponding mean age
Train['age'] = Train.apply(fill_age, axis=1)

# group the dataframe by apache_2_bodysystem and calculate the mean age for each group
mean_age_by_bodysystem = Test.groupby('apache_2_bodysystem')['age'].mean()

# define a function that takes a row of the dataframe as input and returns the mean age of the corresponding apache_2_bodysystem
def fill_age(row):
    if pd.isnull(row['age']):
        if pd.isnull(row['apache_2_bodysystem']):
            return np.nan
        else:
            return mean_age_by_bodysystem[row['apache_2_bodysystem']]
    else:
        return row['age']

# apply the function to each row of the dataframe and fill the missing age values with the corresponding mean age
Test['age'] = Test.apply(fill_age, axis=1)

#for all binary columns we will apply mode imputation for missing values
#first we will create a list of all binary columns
binary_colsTest = ['elective_surgery', 'apache_post_operative', 'gcs_unable_apache', 'intubated_apache', 'ventilated_apache','immunosuppression', 'solid_tumor_with_metastasis']

binary_colsTrain = ['elective_surgery', 'apache_post_operative', 'gcs_unable_apache', 'intubated_apache', 'ventilated_apache','immunosuppression', 'solid_tumor_with_metastasis','hospital_death']
#now we will apply mode imputation on these columns
from sklearn.impute import SimpleImputer
binary_colsTest = [col for col in Train.columns if Train[col].dtype == 'object' or col in binary_colsTest]
binary_colsTrain = [col for col in Test.columns if Test[col].dtype == 'object' or col in binary_colsTrain]

imputer = SimpleImputer(strategy='most_frequent')
Train[binary_colsTrain] = imputer.fit_transform(Train[binary_colsTrain])
Test[binary_colsTest] = imputer.fit_transform(Test[binary_colsTest])

numeric_cols = [col for col in Train.select_dtypes(include=[np.number]).columns if col not in binary_colsTrain]
numeric_colsTest = [col for col in Test.select_dtypes(include=[np.number]).columns if col not in binary_colsTrain]

from sklearn.impute import KNNImputer

# create an instance of KNNImputer with k=5
imputer = KNNImputer(n_neighbors=5)

# fill missing values in Train dataframe
Train[numeric_cols] = imputer.fit_transform(Train[numeric_cols])

# fill missing values in Test dataframe
Test[numeric_colsTest] = imputer.fit_transform(Test[numeric_colsTest])

In [27]:
#do one hot encoding
Train = pd.get_dummies(Train)
Test = pd.get_dummies(Test)

In [28]:
#define x as all columns except target
X=Train.loc[:,Train.columns!='hospital_death']
y=Train['hospital_death']

In [52]:
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.3, random_state=2)

In [65]:
from sklearn.model_selection import GridSearchCV

# define the model
model = DecisionTreeClassifier()

# define the grid of hyperparameters to search
grid = dict()
grid['max_depth'] = [i for i in range(1, 11)]
grid['min_samples_split'] = [i for i in range(2, 101, 2)]

# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='roc_auc')

# execute the grid search
grid_result = grid_search.fit(X, y)

# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best: 0.850439 using {'max_depth': 7, 'min_samples_split': 100}


In [80]:
grid_search.fit(trainX, trainy)

KeyboardInterrupt: 

In [81]:
#now i want to use the best parameters to train my model
model = DecisionTreeClassifier(max_depth=7, min_samples_split=100)
model.fit(trainX, trainy)
#check accurcy using roc
yhat = model.predict_proba(testX)
# keep probabilities for the positive outcome only
yhat = yhat[:, 1]
# calculate roc curves
print(roc_auc_score(testy, yhat))


0.8496458326967409


In [84]:
hospital_death_prob = model.predict_proba(Test)
# The predicted probabilities of each class are stored in the second column of the output array
hospital_death = hospital_death_prob[:, 1]


In [85]:
# Create a DataFrame for the predictions
predictions_df = pd.DataFrame(hospital_death, columns=['hospital_death'])

# Add the record ID from the test data to the predictions DataFrame
predictions_df.insert(0, 'RecordID', Test['RecordID'])

# Save the predictions to a CSV file
predictions_df.to_csv('predictionDT.csv', index=False)

In [78]:
# Create a DataFrame for the predictions
predictions_df = pd.DataFrame(hospital_death, columns=['hospital_death'])

# Add the record ID from the test data to the predictions DataFrame
predictions_df.insert(0, 'RecordID', Test['RecordID'])

# Save the predictions to a CSV file
predictions_df.to_csv('predictionKFOLD.csv', index=False)

In [79]:
predictions_df.shape

(30000, 2)